In [ ]:
import pandas as pd
import pickle
import matplotlib.pyplot as plt
import seaborn as sns
from pandas.plotting import scatter_matrix
from IPython.display import display

from sklearn import preprocessing
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split

3 Factores controlan la intensidad de un sismos a grandes razgos:
- Magnitud
- Distancia del hipocentro
- Suelo

Peak acceleration
- How quickly the ground shook
- How swiftly it changed direction
- Built on mud substrate
  - Bay Mud
  - Allavium
  - Bedrock
Documentr that seismic waves are amplified by a facto of two as they pass from bedrock

In [ ]:
df = pd.read_csv('raw_usa.csv')

In [ ]:
df["properties.time"].min()

In [ ]:
df.shape

In [ ]:
df.info()

cdi y mmi, analizar que hacer con esos pocos valores.

In [ ]:
df.drop(columns=['properties.cdi',	'properties.mmi'], inplace=True)

In [ ]:
df.describe()

In [ ]:
df.head(1)

In [ ]:

df.drop(['type', 'id', 'properties.place', 'properties.time',
       'properties.updated', 'properties.tz', 'properties.url',
       'properties.detail', 'properties.felt', 'properties.status',
       'properties.tsunami', 'properties.net', "properties.magType",
       'properties.code', 'properties.ids', 'properties.sources',
       'properties.nst', 'properties.dmin','properties.title',
         'geometry.type', 'geometry.coordinates', 'properties.rms',
       'Longitud', 'Latitud', 'properties.types'], axis = 1, inplace=True)

Un terremoto no puede producirse físicamente a una profundidad de 0 km o -1 km (por encima de la superficie de la Tierra). Para que se produzca un terremoto, dos bloques de corteza deben deslizarse entre sí, y es imposible que esto ocurra en la superficie terrestre o por encima de ella. Entonces, ¿por qué a veces informamos de que el seísmo se produjo a una profundidad de 0 km o de un suceso como una profundidad negativa?

En primer lugar, la profundidad de un terremoto suele ser la parte más difícil de precisar. Dado que la mayoría de los terremotos se producen a gran profundidad dentro de la corteza terrestre, un error de +/- 1 ó 2 km es irrelevante; en otras palabras, se trata de un pequeño error cuando la profundidad es algo así como 13 km. Sin embargo, si la profundidad del seísmo es relativamente baja, el problema aumenta. Una profundidad negativa puede ser a veces un artefacto de la escasa resolución para un suceso poco profundo.

En el caso de las explosiones de cantera registradas por la red sísmica, la profundidad se fija en 0 km, ya que nunca podemos determinar una profundidad precisa para ellas, pero sabemos que están muy cerca de la superficie.

A veces, debido a la densidad de la red sísmica y a la proximidad de las estaciones sísmicas al epicentro de un seísmo, podemos determinar una profundidad muy precisa. Cuando la profundidad del seísmo es muy baja, puede notificarse como profundidad negativa.

- https://www.usgs.gov/faqs/what-does-it-mean-earthquake-occurred-depth-0-km-how-can-earthquake-have-negative-depth-would

In [ ]:
df['properties.type'].unique()

In [ ]:
df = df.loc[df['properties.type'] != 'quarry blast']

In [ ]:
df.drop(columns='properties.type', inplace=True)

In [ ]:
# Definir un diccionario de mapeo de valores de alerta
mapping = {"green": 1, "yellow": 2, "orange": 3, "red": 4}

# Mapear los valores al DataFrame
df["properties.alert"] = df["properties.alert"].map(mapping)
df["properties.alert"].fillna(0, inplace=True)

In [ ]:
df.info()

En sismología, la profundidad de foco, profundidad focal o profundidad del hipocentro se refiere a la profundidad a la que ocurre un terremoto. Los terremotos que ocurren a una profundidad de menos de 70 kilómetros (43,5 mi) se clasifican como terremotos de foco superficial, mientras que aquellos con una profundidad focal entre 70 kilómetros (43,5 mi) y 300 kilómetros (186,4 mi) se denominan comúnmente terremotos de foco medio o profundidad intermedia.1​ En las zonas de subducción, donde la corteza oceánica más antigua y fría desciende debajo de otra placa tectónica, pueden ocurrir terremotos de foco profundo a profundidades mucho mayores en el manto, que van desde 300 kilómetros (186,4 mi) hasta 700 kilómetros (435 mi).

In [ ]:
rangos_profundida = {
    'Superficial': (-4, 70),
    'Media': (70, 300),
    'Profunda': (300, 700)
}

In [ ]:
df.loc[df["properties.gap"].isna()]
#capaz relacionado con otras fuentes que no sean earthquake, fijarse
df["properties.gap"].fillna(180, inplace=True)

In [ ]:
display(df.shape)
df.dropna(subset='Profundidad', inplace=True)
display(df.shape)

Copio el dataset para el modelo supervisado

In [ ]:
df_supervisado = df.copy()

In [ ]:
df['CategoriaProfundidad'] = pd.cut(df['Profundidad'], bins=[0, 70, 300, 700], labels=['Superficial', 'Media', 'Profunda'], right=False)

In [ ]:
codificacion_profundidad = pd.get_dummies(df['CategoriaProfundidad'])

In [ ]:
df = pd.concat([df, codificacion_profundidad], axis=1)

In [ ]:
map={True: 1, False: 0}
columnas_map = ["Superficial", "Media", "Profunda"]

for columna in columnas_map:
    df[columna] = df[columna].map(map)
    df[columna]
df.drop(columns='CategoriaProfundidad', inplace=True)

In [ ]:
df.head()

In [ ]:
plt.figure(figsize = (10, 7))
cor = df.corr()
sns.heatmap(cor, annot = True, cmap = plt.cm.Reds)
plt.show()

- **.corr()** aplica la correlación de Pearson. Los valores cercanos a cero demuestran una correlacion no lineal.

In [ ]:
# scatter_matrix(df, figsize=(12, 8))
# # Rotar etiquetas en el eje x
# plt.xticks(rotation=90)

# # Rotar etiquetas en el eje y
# plt.yticks(rotation=90)
# plt.savefig('../MACHINE_LEARNING/scatter_matrix.png')
# plt.close()

In [ ]:
# porcentajes = [(80, 20), (85, 15), (90, 10), (95, 5)]
porcentaje = (85, 15)

In [ ]:
# for porcentaje in porcentajes:
# Obtener los porcentajes de entrenamiento y prueba
porcentaje_entrenamiento, porcentaje_prueba = porcentaje

# Calcular el tamaño del conjunto de prueba
porcentaje_prueba = porcentaje_prueba / 100.0

# Dividir el dataframe en conjuntos de entrenamiento y prueba
X_entrenamiento, X_prueba = train_test_split(df, test_size=porcentaje_prueba, random_state=42)
X_entrenamiento = preprocessing.Normalizer().fit_transform(X_entrenamiento)
X_prueba = preprocessing.Normalizer().fit_transform(X_prueba)
lista = []

for i in range(1, 11):
    algoritmo = KMeans(n_clusters = i, init = 'k-means++',
                        max_iter = 300, n_init = 10)
    algoritmo.fit(X_entrenamiento)
    lista.append(algoritmo.inertia_)

plt.figure(figsize = [10, 6])
plt.title('Método de Codo')
plt.xlabel('Nro de clusters')
plt.ylabel('Inercia')
plt.plot(list(range(1, 11)), lista, marker = 'o')
plt.show()

In [ ]:
algoritmo = KMeans(n_clusters = 2, init = 'k-means++', 
                   max_iter = 300, n_init = 10)

algoritmo.fit(X_entrenamiento)

In [ ]:
centroides, etiquetas = algoritmo.cluster_centers_, algoritmo.labels_

muestras_prediccion = algoritmo.predict(X_prueba)

In [ ]:
for i, pred in enumerate(muestras_prediccion):
  print('Muestra', i, ' Clúster:', pred)

In [ ]:
modelo_pca = PCA(n_components = 2)
modelo_pca.fit(X_entrenamiento)
pca = modelo_pca.transform(X_entrenamiento) 
centroides_pca = modelo_pca.transform(centroides)
display(f'Dimensiones de dataset de entrenamiento {X_entrenamiento.shape}')
display(f'Dimensiones de dataset transformado (PCA):{pca.shape}')

In [ ]:
plt.figure(figsize=(7,7))
plt.axes().set_aspect("equal")
plt.scatter(pca[:, 0], [2]*pca[:, 0].size, color='red', alpha=0.5, label='C1')
plt.scatter(pca[:, 1], [1]*pca[:, 1].size, color='blue', alpha=0.5, label='C2')
plt.legend()
plt.show()
display(f'Varianza explicada por cada componente {modelo_pca.explained_variance_}')
display(f'Proporción de varianza explicada por cada componente {modelo_pca.explained_variance_ratio_}')

In [ ]:
# Grafica los resultados
plt.scatter(pca[:, 0], pca[:, 1])
plt.xlabel('Componente Principal 1')
plt.ylabel('Componente Principal 2')
plt.title('PCA en 2 Dimensiones')
plt.show()

In [ ]:
colores = ['red', 'green']

colores_cluster = [colores[etiquetas[i]] for i in range(len(pca))]

plt.scatter(pca[:, 0], pca[:, 1], c = colores_cluster, 
            marker = 'o',alpha = 0.4)

plt.scatter(centroides_pca[:, 0], centroides_pca[:, 1],
            marker = 'x', s = 100, linewidths = 3, c = colores)

xvector = modelo_pca.components_[0] * max(pca[:,0])
yvector = modelo_pca.components_[1] * max(pca[:,1])
columnas = df.columns

for i in range(len(columnas)):
    #Se grafican los vectores
    plt.arrow(0, 0, xvector[i], yvector[i], color = 'black', 
              width = 0.0005, head_width = 0.02, alpha = 0.75)
    plt.text(xvector[i], yvector[i], list(columnas)[i], color='black', 
             alpha=0.75)

plt.show()

**********

***

Clasificacion supervisada

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix, accuracy_score
from matplotlib.colors import ListedColormap

In [ ]:
df_supervisado.head()

In [ ]:
rangos_profundida

probabilidad de peligrosidad = mapa peligrosidad *

In [ ]:
rangos = {(-4, 70): 1, (70, 700): 0}

In [ ]:
df_supervisado['Peligrosidad'] = df_supervisado['Profundidad'].apply(lambda x: next((value for key, value in rangos.items() if key[0] < x <= key[1]), None))

In [ ]:
df_supervisado.head()

In [ ]:
columnas_x = ['properties.mag',	'Profundidad']
columnas_y = ['Peligrosidad']

X = df_supervisado.loc[:, columnas_x]
y = np.ravel(df_supervisado.loc[:, columnas_y])
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

svm_rbf = SVC(kernel='rbf', random_state= 0)
svm_mo = svm_rbf.fit(X_train, y_train)

In [ ]:
# Prediccion
y_pred_svm = svm_rbf.predict(X_test)

In [ ]:
cm = confusion_matrix(y_test, y_pred_svm)
print(cm)
accuracy_score(y_test,y_pred_svm)

In [ ]:
X_set, y_set = X_test, y_test
X1, X2 = np.meshgrid(np.arange(start = X_set[:, 0].min() - 1, stop = X_set[:, 0].max() + 1, step = 0.01),
                     np.arange(start = X_set[:, 1].min() - 1, stop = X_set[:, 1].max() + 1, step = 0.01))
plt.contourf(X1, X2, svm_rbf.predict(np.array([X1.ravel(), X2.ravel()]).T).reshape(X1.shape),
             alpha = 0.75, cmap = ListedColormap(('red', 'green')))
plt.xlim(X1.min(), X1.max())
plt.ylim(X2.min(), X2.max())
for i, j in enumerate(np.unique(y_set)):
    plt.scatter(X_set[y_set == j, 0], X_set[y_set == j, 1],
                color = ListedColormap(('red', 'green'))(i), label = j)
plt.title('SVM (Test set)')
plt.xlabel('Magnitud')
plt.ylabel('Profundidad')
plt.legend()
plt.show()

In [ ]:
# # Combinaciones de dimensiones para graficar
# combinations = [(0, 1), (0, 2), (0, 3), (0, 4), (1, 2), (1, 3), (1, 4), (2, 3), (2, 4), (3, 4)]

# # Configuración de la grilla para los subplots
# grid_rows = 5
# grid_cols = 2

# fig, axs = plt.subplots(grid_rows, grid_cols, figsize=(12, 15))

# # Iterar sobre las combinaciones de dimensiones
# for i, combination in enumerate(combinations):
#     dim1, dim2 = combination
    
#     # Subplot correspondiente
#     ax = axs[i // grid_cols, i % grid_cols]
    
#     X_set = X_test[:, [dim1, dim2]]
#     y_set = y_test

#     # Generar la cuadrícula para graficar
#     X1, X2, X3, X4, X5 = np.meshgrid(
#         np.arange(start=X_set[:, 0].min() - 1, stop=X_set[:, 0].max() + 1, step=0.01),
#         np.arange(start=X_set[:, 1].min() - 1, stop=X_set[:, 1].max() + 1, step=0.01),
#         np.arange(start=X_set[:, 2].min() - 1, stop=X_set[:, 2].max() + 1, step=0.01),
#         np.arange(start=X_set[:, 3].min() - 1, stop=X_set[:, 3].max() + 1, step=0.01),
#         np.arange(start=X_set[:, 4].min() - 1, stop=X_set[:, 4].max() + 1, step=0.01)
#     )

#     # Preparar los datos para la clasificación y graficación
#     X_grid = np.array([X1.ravel(), X2.ravel(), X3.ravel(), X4.ravel(), X5.ravel()]).T
#     y_grid = svm_rbf.predict(X_grid)
#     y_grid = y_grid.reshape(X1.shape)

#     # Graficar los límites de decisión y los puntos de datos
#     ax.contourf(X1, X2, y_grid, alpha=0.75, cmap=ListedColormap(('red', 'green')))
#     ax.set_xlim(X1.min(), X1.max())
#     ax.set_ylim(X2.min(), X2.max())

#     # Graficar los puntos de datos
#     for j, label in enumerate(np.unique(y_set)):
#         ax.scatter(X_set[y_set == label, 0], X_set[y_set == label, 1],
#                    c=ListedColormap(('red', 'green'))(j), label=label)

#     # Configurar el título y etiquetas de los ejes
#     ax.set_title(f"Dimensions {dim1+1} and {dim2+1}")
#     ax.set_xlabel(f"Feature {dim1+1}")
#     ax.set_ylabel(f"Feature {dim2+1}")
#     ax.legend()

# plt.tight_layout()
# plt.show()

# Support Vector machine with poly

In [ ]:
# Definir los parámetros a explorar
parameters = {'kernel': ['poly'], 'degree': [2, 3, 4, 5]}

# Crear el modelo SVM
svm_poly = SVC()

# Realizar la búsqueda de grado utilizando validación cruzada
grid_search = GridSearchCV(svm_poly, parameters, cv=5)
grid_search.fit(X, y)

# Obtener el mejor grado y el modelo con el mejor desempeño
best_degree = grid_search.best_params_['degree']
best_model = grid_search.best_estimator_